In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import io
import os # Importato per gestire percorsi e cartelle

# --- Parametri della Simulazione (per normalizzazione) ---
U_inf = 0.4 # Velocità indisturbata
D = 1.0     # Dimensione caratteristica

# --- Nomi dei file CSV ---
file_x05 = 'line_x0.5.csv'
file_x20 = 'line_x2.0.csv'
file_x30 = 'line_x3.0.csv'
file_cp_base = 'cpMean.csv' # File con CpMean calcolato sulla base

# --- Cartella per i Grafici ---
output_dir = 'plots'
# Crea la cartella se non esiste
os.makedirs(output_dir, exist_ok=True)
print(f"I grafici verranno salvati nella cartella: {output_dir}")

csv_files = {
    "df_x05": file_x05,
    "df_x20": file_x20,
    "df_x30": file_x30,
    "df_cp_base": file_cp_base
}

dataframes = {}
load_error = False

# --- Caricamento Dati ---
print("\n--- Tentativo di Caricamento File CSV ---")
for df_name, filename in csv_files.items():
    full_path = filename # Assume file nella stessa dir dello script
    if not os.path.exists(full_path):
         print(f"\nERRORE: File non trovato -> {filename}")
         print(f"Percorso cercato: {os.path.abspath(full_path)}")
         load_error = True
         break # Interrompi se un file manca

    try:
        try:
            df = pd.read_csv(full_path, delimiter=',', comment='#', encoding='utf-8')
        except UnicodeDecodeError:
            print(f"Warning: Lettura di {filename} con utf-8 fallita, tento con latin-1...")
            df = pd.read_csv(full_path, delimiter=',', comment='#', encoding='latin-1')
        dataframes[df_name] = df
        print(f"Caricato: {filename}")
    except Exception as e:
        print(f"\nERRORE durante il caricamento di {filename}: {e}")
        load_error = True
        break

if load_error:
    print("\nEsecuzione interrotta a causa di errori nel caricamento dei file.")
    raise SystemExit(0)
else:
    print("Tutti i file CSV caricati con successo.")

# --- Assegnazione Variabili DataFrame ---
df_x05 = dataframes["df_x05"]
df_x20 = dataframes["df_x20"]
df_x30 = dataframes["df_x30"]
df_cp_base = dataframes["df_cp_base"]

# --- INDICI DELLE COLONNE (basati sull'output precedente) ---
# !!! VERIFICA CHE QUESTI INDICI SIANO ANCORA CORRETTI PER I TUOI FILE !!!
# File line_*.csv:
col_y_idx = 9        # Indice per 'Points:1'
col_pMean_idx = 7    # Indice per 'pMean'
col_UMean_X_idx = 20 # Indice per 'UMean:0'
col_UMean_Y_idx = 21 # Indice per 'UMean:1'
# File cpMean.csv:
col_cpMean_idx = 3   # Indice per 'cpMean'
col_y_base_idx = 10  # Indice per 'Points:1' nel file cpMean

# --- Creazione Grafici (usando .iloc) ---
print("\n--- Inizio Creazione e Salvataggio Grafici ---")
try:
    # Dizionario per ciclare sui file delle linee
    line_data = {
        'x/D = 0.5': (df_x05, 'x0.5'),
        'x/D = 2.0': (df_x20, 'x2.0'),
        'x/D = 3.0': (df_x30, 'x3.0')
    }

    for label, (df, suffix) in line_data.items():
        print(f"Processo linea {label}...")

        # --- Grafico pMean ---
        plt.figure(figsize=(8, 6))
        plt.plot(df.iloc[:, col_pMean_idx], df.iloc[:, col_y_idx])
        plt.xlabel('$p_{Mean} / \\rho$  $[m^2/s^2]$')
        plt.ylabel('$y/D$')
        plt.title(f'Profilo pMean a {label}')
        plt.grid(True)
        plt.ylim([-2, 2])
        plt.tight_layout()
        filename = os.path.join(output_dir, f'pMean_{suffix}.png')
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"  Grafico salvato in: {filename}")
        plt.close()

        # --- Grafico U/V ---
        plt.figure(figsize=(8, 6))
        df_cleaned = df.dropna(subset=[df.columns[col_UMean_X_idx], df.columns[col_UMean_Y_idx], df.columns[col_y_idx]])
        plt.plot(df_cleaned.iloc[:, col_UMean_X_idx] / U_inf, df_cleaned.iloc[:, col_y_idx] / D, label='$U / U_{\\infty}$', linestyle='-')
        plt.plot(df_cleaned.iloc[:, col_UMean_Y_idx] / U_inf, df_cleaned.iloc[:, col_y_idx] / D, label='$V / U_{\\infty}$', linestyle='--')
        plt.xlabel('Velocità Normalizzata')
        plt.ylabel('$y/D$')
        plt.title(f'Profili Velocità Medie Normalizzate a {label}')
        plt.legend()
        plt.grid(True)
        plt.ylim([-2, 2])
        plt.axvline(0, color='black', linewidth=0.5)
        plt.tight_layout()
        filename = os.path.join(output_dir, f'U_V_{suffix}.png')
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"  Grafico salvato in: {filename}")
        plt.close()

    # --- Grafico CpMean sulla base (MODIFICATO per asse Y = y/D) ---
    print("Processo linea base (CpMean)...")
    col_CpMean_name = None # Inizializza
    if col_cpMean_idx < len(df_cp_base.columns):
        col_CpMean_name = df_cp_base.columns[col_cpMean_idx]
        print(f"  Trovata colonna CpMean: '{col_CpMean_name}'")
    elif col_pMean_idx < len(df_cp_base.columns):
         col_pMean_name = df_cp_base.columns[col_pMean_idx]
         print(f"ATTENZIONE: Colonna '{df_cp_base.columns[col_cpMean_idx]}' non trovata. Calcolo da '{col_pMean_name}'.")
         df_cp_base['CpMean_calc'] = df_cp_base.iloc[:, col_pMean_idx] / 0.08 # Calcola Cp
         col_CpMean_name = 'CpMean_calc' # Usa il nome della colonna calcolata
    else:
        print(f"ERRORE: Non trovata né la colonna Cp all'indice {col_cpMean_idx} né la colonna pMean all'indice {col_pMean_idx} in {file_cp_base}.")

    # Procedi solo se abbiamo un nome valido per la colonna Cp e per la colonna Y
    if col_CpMean_name and col_y_base_idx < len(df_cp_base.columns):
        col_y_base_name = df_cp_base.columns[col_y_base_idx] # Prende il nome Y dall'indice
        print(f"  Trovata colonna Y: '{col_y_base_name}'")

        # Calcola y/D (invece di 2y/D)
        df_cp_base['y_D'] = df_cp_base.iloc[:, col_y_base_idx] / D
        # Rimuovi eventuali righe con NaN prima di plottare
        df_cp_base_cleaned = df_cp_base.dropna(subset=[col_CpMean_name, 'y_D'])

        plt.figure(figsize=(6, 8))
        plt.plot(df_cp_base_cleaned[col_CpMean_name], df_cp_base_cleaned['y_D']) # Usa y_D
        plt.ylabel('$y/D$') # Etichetta modificata
        plt.xlabel('$C_p$')
        plt.title('Profilo Cp Medio sulla Base (x/D = 1.0)')
        plt.grid(True)
        plt.xlim([-1.6, 0.4]) # Limiti X invariati
        plt.ylim([-0.6, 0.6]) # Limiti Y modificati per scala y/D
        plt.tight_layout()
        filename = os.path.join(output_dir, 'Cp_base_vs_yD.png') # Nome file leggermente modificato
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"  Grafico salvato in: {filename}")
        plt.close()
    else:
         print(f"ERRORE: La colonna Y all'indice {col_y_base_idx} non è valida in {file_cp_base}. Impossibile creare il grafico Cp.")


except IndexError as e:
     print(f"\nERRORE DI INDICE: {e}")
     print("L'indice numerico della colonna specificato non è valido per uno dei DataFrame.")
     print("Verifica gli indici col_*_idx nello script e assicurati che i file CSV non siano cambiati.")
except Exception as e:
    print(f"Si è verificato un errore durante la creazione dei grafici: {e}")

print("\nScript completato.")

I grafici verranno salvati nella cartella: plots

--- Tentativo di Caricamento File CSV ---
Caricato: line_x0.5.csv
Caricato: line_x2.0.csv
Caricato: line_x3.0.csv
Caricato: cpMean.csv
Tutti i file CSV caricati con successo.

--- Inizio Creazione e Salvataggio Grafici ---
Processo linea x/D = 0.5...
  Grafico salvato in: plots/pMean_x0.5.png
  Grafico salvato in: plots/U_V_x0.5.png
Processo linea x/D = 2.0...
  Grafico salvato in: plots/pMean_x2.0.png
  Grafico salvato in: plots/U_V_x2.0.png
Processo linea x/D = 3.0...
  Grafico salvato in: plots/pMean_x3.0.png
  Grafico salvato in: plots/U_V_x3.0.png
Processo linea base (CpMean)...
  Trovata colonna CpMean: 'cpMean'
  Trovata colonna Y: 'Points_1'
  Grafico salvato in: plots/Cp_base_vs_yD.png

Script completato.
